In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from unet.unet_model_keras import UNet
import os
import sys
import skimage.io
from pathlib import Path
#import keras stuff? maybe

In [ ]:
tf.__version__

In [ ]:
videoPath = '../data/videos_croppedFrames/'
videoNum = 'demo_025.mp4'

In [ ]:
cap = cv2.VideoCapture(os.path.join(videoPath, videoNum))
frameList = list()
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
 
    # Display the resulting frame
#     cv2.imshow('Frame',frame)

    res_frame = cv2.resize(frame, (384, 384), interpolation = cv2.INTER_AREA)
    
    frameList.append(res_frame)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
#   Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
def gen_frame_sequence(seqlen):
#     print(seqlen)
    
    sequences = list()
    
    for i in range(449): #change to 449 later
        tmpseq = list()
        for j in range(seqlen//2):
            k = i - (seqlen//2  - j)
            tmpseq.append(frameList[k] if k >=0 else frameList[(k + (int(abs(k))))])
        tmpseq.append(frameList[i])
        for j in range(seqlen//2):
            k = i + j + 1
            tmpseq.append(frameList[k] if k <=448 else frameList[(k - (int(abs(k - 448))))])
        sequences.append(np.array(tmpseq))
    return sequences


In [ ]:
# weightsPath = os.path.abspath('/home/omkar/ArteryProj/test_models/model_060_1')
# unet_model.load_weights(weightsPath)
# print(weightsPath)

In [ ]:
model_dir = os.path.join(Path().resolve(), 'test_models/UNetSeq')
print(model_dir)

In [ ]:
model_dir = os.path.join(Path().resolve(),'')
seqlen = 3
print(model_dir)

In [ ]:
unet_model = tf.keras.models.load_model(os.path.join(model_dir, 'checkpoints/model_001'))
# unet_model = tf.keras.models.load_model(os.path.join(model_dir, 'runs_slen_7/model_080'))
# unet_model = tf.keras.models.load_model('/home/omkar/ArteryProj/test_models/model_060_2')
# print(unet_model.summary())

In [ ]:
sequences = gen_frame_sequence(seqlen)
print(len(sequences))

In [ ]:
def postprocess(img, mask):

    vimg = np.asarray(img, np.uint8)
    vmask = np.asarray(mask, np.uint8)
#     vmask = np.expand_dims(vmask, axis = 2)
#     vmaskf = np.append(vmask, vmask, axis = 2)
#     vmaskf = np.append(vmaskf, vmask, axis = 2)
    vmask = vmask * [0, 0, 127]
    # print(vmaskf)
    # print(vimg)
    alpha = 0.6

#     print(vmask.shape, vimg.shape)

    vop = np.zeros(shape = vimg.shape, dtype=np.uint8)

    cv2.addWeighted(np.asarray(vmask, np.uint8), alpha, vimg, 1 - alpha, 0, vop)

    return vop

In [ ]:
def create_mask(pred_mask):
    
    pred_mask = np.array(pred_mask)
    pred_mask = pred_mask[0] > 0.85
    
    pred_mask = (pred_mask) * 1
    
#     pred_mask = tf.math.argmin(pred_mask, axis=-1)
#     pred_mask = pred_mask[..., tf.newaxis]
#     print("pred_mask shape: ", pred_mask.shape)
    return pred_mask

In [ ]:
len(sequences[0])

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'avc1')
outname = 'unetseq_' + videoNum
out = cv2.VideoWriter(outname, fourcc, 25, (384, 384))

In [ ]:
ctr = 0
for seq in sequences:
    
    
    with np.printoptions(threshold=sys.maxsize):

    #     images = list()

    #     for fname in seq:

    #         tpimg = skimage.io.imread('/home/omkar/ArteryProj/data/Imgs_demo031_squared/' + str(fname) + '.png')
    #         images.append(tpimg)

    #     images = np.array(images)
        #expand dims if necessary
        img = seq[seqlen//2 + 1]
    #     print("img shape: ", img.shape)
    #     images = np.expand_dims(images, axis = 0)
        seq = np.expand_dims(seq, axis = 0)
    #     print("images shape: ", images.shape)
    #     mask = unet_model(images, training=False)
        mask = unet_model(seq, training=False)
#         print("mask shape:", mask.shape)
#         print("Mask:", np.squeeze(mask))
        mask = create_mask(mask)
    #     print("created mask shape: ", mask.shape)
    #     print("mask shape:", mask.shape)
    #     with np.printoptions(threshold=np.inf):
    #         for i in mask:
    #             for j in i:
    #                 print(j)
        img = postprocess(img, mask)

    #     mask = np.asarray(mask, np.uint8)
    #     mask = mask * [0, 0, 255]

        ctr += 1
        if(ctr%10 == 0):
            print(ctr)

        out.write(img)

#         if(ctr > 0):
#             break
out.release()        